In [1]:
from pyfaidx import Fasta

In [6]:
# поменять на свой
fasta_file_path = 'data_dir/GRCh38.primary_assembly.genome.fa.gz'

In [7]:
fasta = Fasta(fasta_file_path)

In [8]:
chromosome = 'chr18'
position_of_interest = 47245

In [26]:
sequence_of_interest = fasta[chromosome][position_of_interest - 4:position_of_interest + 11]

In [27]:
print(sequence_of_interest)

ATTAGTCAAAACCGC


In [31]:
for i, nucleotide in enumerate(sequence_of_interest, 47242):
    print(f"{i} - {nucleotide}")

47242 - A
47243 - T
47244 - T
47245 - A
47246 - G
47247 - T
47248 - C
47249 - A
47250 - A
47251 - A
47252 - A
47253 - C
47254 - C
47255 - G
47256 - C


![nucleotide-positions.png](https://i.postimg.cc/prWyGB5Y/nucleotide-positions.png)